<a href="https://colab.research.google.com/github/LotusBro98/Lanat2019/blob/master/Lanat2019/Lessons/Machine_Learning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинное обучение
## Занятие 1. Основы
### [Open in Google Colab](https://colab.research.google.com/github/LotusBro98/Lanat2019/blob/master/Lessons/Machine_Learning_1.ipynb#scrollTo=_lm7unL06NFP)

---

<table>
  <tr>
    <td width="50%" rowspan="2"><img src="https://pm1.narvii.com/6848/8588abbd8f81f22e0b728beb01ad5351501fd867v2_hq.jpg"></td>
    <td width="50%"><img src="https://i.pinimg.com/originals/ec/7c/6b/ec7c6b7cb668bdb7a419c9ce9d8969d1.png"></td>
  </tr>
  <tr><td height="100%"></td></tr>
</table>

---

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from typing import List

Допустим, существует некая закономерность, по которой можно отделить мальчиков от девочек, знае рост и вес. 
Пусть этот "закон природы" выражается с  помощью математической функции (она нам неизвестна, но мы предполагаем, что она есть)

На этом занятии будет показано, как с помощью машинного обучения найти неизвестную закономерность по экспериментальным данным.

In [0]:
# Рост и  вес приведены к масштабу от 0 до 1
def real_chan_kun_func(h: np.ndarray, w: np.ndarray) -> np.ndarray:
  return np.clip((h - w) * 10, -1, 1)

Построим график распределения "закона природы"<br/>
Синий - область, соответствующая мальчикам,<br/>
Красный - девочкам,<br/>
Белый переход - зона неопределенности

In [0]:
def get_distribution(func, N=25):
  xs = np.linspace(1, 0, N)
  ys = np.linspace(1, 0, N)

  xs = np.repeat([xs], N, axis=0)
  ys = np.repeat(np.transpose([ys]), N, axis=1)

  dist = func(xs, ys)
  return dist

dist = get_distribution(real_chan_kun_func)
plt.xlabel("Рост")
plt.ylabel("Вес")
plt.imshow(dist, origin="lower", cmap="RdBu", interpolation='nearest', extent=(0, 1, 0, 1))
plt.colorbar()
plt.show()

## Сбор данных

In [0]:
# Создадим группу из N человек
N_PEOPLE = 20

weights = np.random.randint(10, 100, N_PEOPLE)
heights = np.random.randint(100, 200, N_PEOPLE)

ax = plt.subplot()
ax.plot(heights, weights, 'o')
plt.xlabel("Рост")
plt.ylabel("Вес")
plt.show()

Теперь представим, что мы спросили каждого человека, кто он, и закон выполнялся.

In [0]:
# Приводим к одному виду
w = weights / 100
h = heights / 100 - 1

# Определяем, мальчик или девочка по заданной функции
chan_kuns_genders = real_chan_kun_func(h, w)
chan_kuns = list(zip(w, h, chan_kuns_genders))

# Отделяем девочек от мальчиков
chans = list(filter(lambda man: man[2] >= 0, chan_kuns))
kuns =  list(filter(lambda man: man[2] <  0, chan_kuns))

# Сохраняем отдельно вес и рост
chans_weights = list(map(lambda chan: chan[0], chans))
chans_heights = list(map(lambda chan: chan[1], chans))

# Сохраняем отдельно вес и рост
kuns_weights = list(map(lambda kun: kun[0], kuns))
kuns_heights = list(map(lambda kun: kun[1], kuns))

# Рисуем график
ax = plt.subplot()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
plt.xlabel("Рост")
plt.ylabel("Вес")
ax.plot(kuns_heights, kuns_weights, 'bo')
ax.plot(chans_heights, chans_weights, 'ro')
plt.show()

## Построение модели

Теперь попробуем построить модель, которая будет приближаться к настоящему закону. <br/><br/>
Построим модель 1 нейрона с 2 входами

f(x, y) = x *  wx + y * wy + b

In [0]:
class Neuron():
  weight_x = np.random.normal()
  weight_y = np.random.normal()
  bias = np.random.normal()
  activation = np.tanh
  
  def __call__(self, x, y):
    return self.activation(x * self.weight_x + y * self.weight_y + self.bias)
  
model = Neuron()

Посмотрим на график распределения необученного нейрона

In [0]:
dist = get_distribution(model)
plt.xlabel("Рост")
plt.ylabel("Вес")
plt.imshow(dist, origin="lower", cmap="RdBu", interpolation='nearest', extent=(0, 1, 0, 1))
plt.colorbar()
plt.show()

Чтобы обучить наш нейрон, используем известные экспериментальные данные.

## Обучение модели

Для начала научим нашу модель оценивать саму себя.
Тогда она сможет обучаться автоматически, чтобы улучшить эту оценку. <br/>
Определим функцию потерь (loss function), которая показывает, насколько сильно отличается истинное распределение от предсказываемого.

In [0]:
# Среднеквадратичное отклонение
def loss(genders_real: np.ndarray, genders_predicted: np.ndarray):
  sqdiff = np.square(genders_real - genders_predicted)
  avg = np.average(sqdiff,  axis=-1)
  return np.sqrt(avg)

In [0]:
# теперь оценим нашу функцию

def loss_process(func):
  return loss(chan_kuns_genders, func(h, w))

loss_neuron = loss_process(model)

print(loss_neuron)

Попробуем немного улучшить оценку, подправив внутренние параметры нейрона 

In [0]:
def train_step(neuron, delta = 0.001, learning_rate=0.5):
  # "Тупо" меняем параметры и смотрим, насколько улучшилась оценка, а затем меняем параметры в сторону улучшения.
  # На практике используют метод градиентного спуска, суть его примерно такая же, но алгоритм вычисления намного более эффективен.
  
  L = loss_process(neuron)
  neuron.weight_x += delta
  L_dx = loss_process(neuron)
  neuron.weight_x -= delta
  
  neuron.weight_y += delta
  L_dy = loss_process(neuron)
  neuron.weight_y -= delta
  
  neuron.bias += delta
  L_db = loss_process(neuron)
  neuron.bias -= delta
  
  dL_dx = (L_dx - L) / delta
  dL_dy = (L_dy - L) / delta
  dL_db = (L_db - L) / delta
  
  neuron.weight_x -= dL_dx * learning_rate
  neuron.weight_y -= dL_dy * learning_rate
  neuron.bias     -= dL_db * learning_rate

train_step(model)
print(loss_process(model))

Как можно видеть, оценка улучшилась (значение функции потерь стало меньше). Посмторим,  что стало с графиком распределениея нашего нейрона.

In [0]:
dist = get_distribution(model)
plt.xlabel("Рост")
plt.ylabel("Вес")
plt.imshow(dist, origin="lower", cmap="RdBu", interpolation='nearest', extent=(0, 1, 0, 1))
plt.colorbar()
plt.show()

Он немного сместился в нужном направлении. <br/>

Теперь запустим обучение в цикле.

In [0]:
%%capture
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation
plt.rcParams["animation.html"] = "jshtml"
N_FRAMES = 100

#model = neuron()

fig, ax = plt.subplots()
dist_plot = ax.imshow([[]], origin="lower", cmap="RdBu", interpolation='nearest', extent=(0, 1, 0, 1))
ax.plot(kuns_heights, kuns_weights, 'bo')
ax.plot(chans_heights, chans_weights, 'ro')
plt.xlabel("Рост")
plt.ylabel("Вес")
plt.show()

def animation_step(i):
  train_step(model)
  dist = get_distribution(model)
  dist_plot.set_data(dist)
  
  
ani = matplotlib.animation.FuncAnimation(fig, animation_step, frames=N_FRAMES)

In [0]:
ani

# А теперь поиграем в песочнице!

## [playground.tensorflow.org](http://playground.tensorflow.org)